# **Regression**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* outputs/datasets/cleaned/BTCAugmented.csv

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/bitcoin-forecast/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/bitcoin-forecast'

# Load Cleaned Data

Section 1 content

In [4]:
import numpy as np
import pandas as pd
df = (pd.read_csv("outputs/datasets/cleaned/BTCAugmented.csv", index_col='date', parse_dates=True)
      .drop(labels=['buy/sell'], axis=1)
      
      )

print(df.shape)
df.head(5)

(2650, 17)


,open,high,low,close,Volume BTC,Volume USD,price mean,upper shadow,lower shadow,spread,trade,10 period SMA,50 period SMA,100 period SMA,12EMA,26EMA,MACD
date,,,,,,,,,,,,,,,,,
2014-11-29,376.42,386.60,372.25,376.72,2746157.05,7245.19,377.9975,9.88,4.17,14.35,0.30,0.0,0.0,0.0,376.347692,376.312593,0.035100
2014-11-30,376.57,381.99,373.32,373.34,1145566.61,3046.33,376.3050,5.42,0.02,8.67,-3.23,0.0,0.0,0.0,375.884970,376.092401,-0.207430
2014-12-01,376.40,382.31,373.03,378.39,2520662.37,6660.56,377.5325,3.92,3.37,9.28,1.99,0.0,0.0,0.0,376.270360,376.262593,0.007766
2014-12-02,378.39,382.86,375.23,379.25,2593576.46,6832.53,378.9325,3.61,3.16,7.63,0.86,0.0,0.0,0.0,376.728766,376.483883,0.244883
2014-12-03,379.25,383.00,374.23,376.67,2998357.92,7908.29,378.2875,3.75,2.44,8.77,-2.58,0.0,0.0,0.0,376.719725,376.497669,0.222056


---

# MP Pipeline: Regressor

## Create ML pipeline

In [5]:
from sklearn.pipeline import Pipeline
from feature_engine import transformation as vt

# Feature Engineering
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection

# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor


def PipelineOptimization(model):
    pipeline_base = Pipeline([

        ('numerical_transformation', vt.YeoJohnsonTransformer(variables=['open', 'high', 'low', 'Volume BTC', 'Volume USD', 
                         'price mean', 'upper shadow', 'lower shadow', 
                         'spread', '10 period SMA', '50 period SMA', 
                         '100 period SMA', '12EMA', '26EMA'])),


        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None,
         method="spearman", threshold=0.6, selection_method="variance")),

        ("feat_scaling", StandardScaler()),

        ("feat_selection",  SelectFromModel(model)),

        ("model", model),

    ])

    return pipeline_base

Custom Class for hyperparameter optimisation

In [6]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

## Split Train Test Set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['close'], axis=1),
    df['close'],
    test_size=0.2,
    random_state=0
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

* Train set: (2120, 16) (2120,) 
* Test set: (530, 16) (530,)


## Grid Search CV - Sklearn

Use default hyperparameters to find most suitable algorithm

In [8]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

Do a hyperparameter optimisation search using default hyperparameters

In [9]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)


Running GridSearchCV for LinearRegression 

Fitting 5 folds for each of 1 candidates, totalling 5 fits

Running GridSearchCV for DecisionTreeRegressor 

Fitting 5 folds for each of 1 candidates, totalling 5 fits

Running GridSearchCV for RandomForestRegressor 

Fitting 5 folds for each of 1 candidates, totalling 5 fits

Running GridSearchCV for ExtraTreesRegressor 

Fitting 5 folds for each of 1 candidates, totalling 5 fits

Running GridSearchCV for AdaBoostRegressor 

Fitting 5 folds for each of 1 candidates, totalling 5 fits

Running GridSearchCV for GradientBoostingRegressor 

Fitting 5 folds for each of 1 candidates, totalling 5 fits

Running GridSearchCV for XGBRegressor 

Fitting 5 folds for each of 1 candidates, totalling 5 fits


Check results

In [10]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

,estimator,min_score,mean_score,max_score,std_score
5,GradientBoostingRegressor,0.735414,0.774869,0.819045,0.035448
2,RandomForestRegressor,0.706472,0.75856,0.805911,0.038643
3,ExtraTreesRegressor,0.706407,0.749811,0.799592,0.035636
6,XGBRegressor,0.605743,0.703395,0.782389,0.058332
1,DecisionTreeRegressor,0.539981,0.609183,0.650462,0.040267
4,AdaBoostRegressor,0.353095,0.442968,0.497109,0.048486
0,LinearRegression,0.328743,0.371711,0.394957,0.022542


## Extensive search on the most suitable model to find the best hyperparameter configuration.

Define model and parameters, for Extensive Search

In [11]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

models_search = {
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
}

# documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html


params_search = {
    "GradientBoostingRegressor": {
        'model__n_estimators': [100,300],
        'model__learning_rate': [1e-1,1e-2,1e-3], 
        'model__max_depth': [3,10,None],
    }
}

Extensive GridSearch CV

In [12]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)


Running GridSearchCV for GradientBoostingRegressor 

Fitting 5 folds for each of 18 candidates, totalling 90 fits


Check results

In [13]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

,estimator,min_score,mean_score,max_score,std_score,model__learning_rate,model__max_depth,model__n_estimators
0,GradientBoostingRegressor,0.735414,0.774869,0.819045,0.035448,0.1,3,100
7,GradientBoostingRegressor,0.730374,0.772034,0.819951,0.035083,0.01,3,300
1,GradientBoostingRegressor,0.700761,0.750752,0.788642,0.03538,0.1,3,300
9,GradientBoostingRegressor,0.655255,0.714792,0.752572,0.039547,0.01,10,300
2,GradientBoostingRegressor,0.652529,0.705284,0.746779,0.039252,0.1,10,100
3,GradientBoostingRegressor,0.650651,0.704027,0.74518,0.03933,0.1,10,300
8,GradientBoostingRegressor,0.598648,0.651364,0.704174,0.037704,0.01,10,100
6,GradientBoostingRegressor,0.59622,0.638259,0.671159,0.030272,0.01,3,100
11,GradientBoostingRegressor,0.561086,0.637423,0.67256,0.040121,0.01,None,300
4,GradientBoostingRegressor,0.545081,0.622965,0.65645,0.041337,0.1,None,100


Check the best model

In [14]:
best_model = grid_search_summary.iloc[0, 0]
best_model

'GradientBoostingRegressor'

Parameters for best model

In [16]:
grid_search_pipelines[best_model].best_params_

{'model__learning_rate': 0.1,
 'model__max_depth': 3,
 'model__n_estimators': 100}

Define the best regressor, based on search

In [17]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

Pipeline(steps=[('numerical_transformation',
                 YeoJohnsonTransformer(variables=['open', 'high', 'low',
                                                  'Volume BTC', 'Volume USD',
                                                  'price mean', 'upper shadow',
                                                  'lower shadow', 'spread',
                                                  '10 period SMA',
                                                  '50 period SMA',
                                                  '100 period SMA', '12EMA',
                                                  '26EMA'])),
                ('SmartCorrelatedSelection',
                 SmartCorrelatedSelection(method='spearman',
                                          selection_method='variance',
                                          threshold=0.6)),
                ('feat_scaling', StandardScaler()),
                ('feat_selection',
                 SelectFromModel(estimator=GradientBoostingRegressor(random_state=0))),
                ('model', GradientBoostingRegressor(random_state=0))])

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
